In [1]:
import pandas as pd
import os
from glob import glob
import exifread
import re

In [2]:
# Ref: https://gist.github.com/snakeye/fdc372dbf11370fe29eb

def _get_if_exist(data, key):
    if key in data:
        return data[key]

    return None


def _convert_to_degress(value):
    """
    Helper function to convert the GPS coordinates stored in the EXIF to degress in float format
    :param value:
    :type value: exifread.utils.Ratio
    :rtype: float
    """
    d = float(value.values[0].num) / float(value.values[0].den)
    m = float(value.values[1].num) / float(value.values[1].den)
    s = float(value.values[2].num) / float(value.values[2].den)

    return d + (m / 60.0) + (s / 3600.0)
    
def get_exif_location(exif_data):
    """
    Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)
    """
    lat = None
    lon = None

    gps_latitude = _get_if_exist(exif_data, 'GPS GPSLatitude')
    gps_latitude_ref = _get_if_exist(exif_data, 'GPS GPSLatitudeRef')
    gps_longitude = _get_if_exist(exif_data, 'GPS GPSLongitude')
    gps_longitude_ref = _get_if_exist(exif_data, 'GPS GPSLongitudeRef')

    if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
        lat = _convert_to_degress(gps_latitude)
        if gps_latitude_ref.values[0] != 'N':
            lat = 0 - lat

        lon = _convert_to_degress(gps_longitude)
        if gps_longitude_ref.values[0] != 'E':
            lon = 0 - lon

    return lat, lon

In [44]:
count = 0
out = []
lat = None
lon = None
_id = None
for dn in glob('/opt/count_women/*'):
    bn = os.path.basename(dn)
    m = re.match('(.*)-(.*)-(.*)', bn)
    if m:
        lat, lon, _id = m.group(1), m.group(2), m.group(3)
    for fn in glob(dn + '/*.jpg'):
        print fn
        r = {'lat': lat, 'long': lon, 'id': _id, 'filename': fn, 'url': bn + '/' + os.path.basename(fn)}
        # Open image file for reading (binary mode)
        xlat = None
        xlon = None
        with open(fn, 'rb') as f:
            # Return Exif tags
            tags = exifread.process_file(f)
            if True:
                for t in tags.keys():
                    #if t.startswith('GPS'):
                    if t != 'JPEGThumbnail':
                        #print t, '==>', tags[t]
                        nt = '_' + t.replace(' ', '_')
                        r[nt] = tags[t]
            xlat, xlon = get_exif_location(tags)
            r['exif_lat'] = xlat
            r['exif_long'] = xlon
            print '-'*50
        out.append(r)
        count += 1
        #break
    #if count  > 3:
    #    break
print count

/opt/count_women/28.5859952-77.0453363-1215/1480758424737.jpg
--------------------------------------------------
/opt/count_women/28.5859952-77.0453363-1215/1480758665273.jpg
--------------------------------------------------
/opt/count_women/28.5859952-77.0453363-1215/1480758906950.jpg
--------------------------------------------------
/opt/count_women/28.5859952-77.0453363-1215/1480758368665.jpg
--------------------------------------------------
/opt/count_women/28.5859952-77.0453363-1215/1480758784165.jpg
--------------------------------------------------
/opt/count_women/28.5859952-77.0453363-1215/1480758724574.jpg
--------------------------------------------------
/opt/count_women/28.5859952-77.0453363-1215/1480758845844.jpg
--------------------------------------------------
/opt/count_women/28.5859952-77.0453363-1215/1480758487028.jpg
--------------------------------------------------
/opt/count_women/28.5859952-77.0453363-1215/1480758604217.jpg
----------------------------------

In [41]:
df = pd.DataFrame(out)
df['exif_date'] = pd.to_datetime(df['_EXIF_DateTimeOriginal'], format="%Y:%m:%d %H:%M:%S").dt.date
df['exif_time'] = pd.to_datetime(df['_EXIF_DateTimeOriginal'], format="%Y:%m:%d %H:%M:%S").dt.time

fixed_cols = ['lat', 'long', 'id', 'filename', 'exif_lat', 'exif_long', 'exif_date', 'exif_time', 'url']
cols = fixed_cols
for c in df.columns:
    if c not in cols:
        cols.append(c)
df = df[cols]

In [42]:
df

,lat,long,id,filename,exif_lat,exif_long,exif_date,exif_time,url,_EXIF_ApertureValue,...,_Interoperability_InteroperabilityIndex,_Interoperability_InteroperabilityVersion,_Thumbnail_Compression,_Thumbnail_JPEGInterchangeFormat,_Thumbnail_JPEGInterchangeFormatLength,_Thumbnail_Orientation,_Thumbnail_ResolutionUnit,_Thumbnail_XResolution,_Thumbnail_YCbCrPositioning,_Thumbnail_YResolution
0,28.5859952,77.0453363,1215,/opt/count_women/28.5859952-77.0453363-1215/14...,NaN,NaN,2016-12-03,15:17:02,28.5859952-77.0453363-1215/1480758424737.jpg,NaN,...,R98,"[48, 49, 48, 48]",JPEG (old-style),1229,14592,Horizontal (normal),Pixels/Inch,72,Co-sited,72
1,28.5859952,77.0453363,1215,/opt/count_women/28.5859952-77.0453363-1215/14...,NaN,NaN,2016-12-03,15:21:03,28.5859952-77.0453363-1215/1480758665273.jpg,NaN,...,R98,"[48, 49, 48, 48]",JPEG (old-style),1229,14848,Horizontal (normal),Pixels/Inch,72,Co-sited,72
2,28.5859952,77.0453363,1215,/opt/count_women/28.5859952-77.0453363-1215/14...,NaN,NaN,2016-12-03,15:25:05,28.5859952-77.0453363-1215/1480758906950.jpg,NaN,...,R98,"[48, 49, 48, 48]",JPEG (old-style),1229,14080,Horizontal (normal),Pixels/Inch,72,Co-sited,72
3,28.5859952,77.0453363,1215,/opt/count_women/28.5859952-77.0453363-1215/14...,NaN,NaN,2016-12-03,15:16:06,28.5859952-77.0453363-1215/1480758368665.jpg,NaN,...,R98,"[48, 49, 48, 48]",JPEG (old-style),1229,12544,Horizontal (normal),Pixels/Inch,72,Co-sited,72
4,28.5859952,77.0453363,1215,/opt/count_women/28.5859952-77.0453363-1215/14...,NaN,NaN,2016-12-03,15:23:02,28.5859952-77.0453363-1215/1480758784165.jpg,NaN,...,R98,"[48, 49, 48, 48]",JPEG (old-style),1229,14592,Horizontal (normal),Pixels/Inch,72,Co-sited,72
5,28.5859952,77.0453363,1215,/opt/count_women/28.5859952-77.0453363-1215/14...,NaN,NaN,2016-12-03,15:22:03,28.5859952-77.0453363-1215/1480758724574.jpg,NaN,...,R98,"[48, 49, 48, 48]",JPEG (old-style),1229,14336,Horizontal (normal),Pixels/Inch,72,Co-sited,72
6,28.5859952,77.0453363,1215,/opt/count_women/28.5859952-77.0453363-1215/14...,NaN,NaN,2016-12-03,15:24:02,28.5859952-77.0453363-1215/1480758845844.jpg,NaN,...,R98,"[48, 49, 48, 48]",JPEG (old-style),1229,14080,Horizontal (normal),Pixels/Inch,72,Co-sited,72
7,28.5859952,77.0453363,1215,/opt/count_women/28.5859952-77.0453363-1215/14...,NaN,NaN,2016-12-03,15:18:04,28.5859952-77.0453363-1215/1480758487028.jpg,NaN,...,R98,"[48, 49, 48, 48]",JPEG (old-style),1229,14464,Horizontal (normal),Pixels/Inch,72,Co-sited,72
8,28.5859952,77.0453363,1215,/opt/count_women/28.5859952-77.0453363-1215/14...,NaN,NaN,2016-12-03,15:20:02,28.5859952-77.0453363-1215/1480758604217.jpg,NaN,...,R98,"[48, 49, 48, 48]",JPEG (old-style),1229,14720,Horizontal (normal),Pixels/Inch,72,Co-sited,72
9,28.5859952,77.0453363,1215,/opt/count_women/28.5859952-77.0453363-1215/14...,NaN,NaN,2016-12-03,15:19:03,28.5859952-77.0453363-1215/1480758544956.jpg,NaN,...,R98,"[48, 49, 48, 48]",JPEG (old-style),1229,14336,Horizontal (normal),Pixels/Inch,72,Co-sited,72


In [43]:
df.to_csv('/opt/count_women/output.csv', index_label='no')